In [1]:
!gcloud services enable aiplatform.googleapis.com

In [17]:
%%bash
gcloud compute firewall-rules create allow-imc-streamlit \
    --direction=INGRESS \
    --priority=1000 \
    --network=default \
    --action=ALLOW \
    --rules=tcp:8501 \
    --source-ranges=0.0.0.0/0 2> /dev/null

gcloud compute firewall-rules list --filter name='allow-imc-streamlit' 2> /dev/null

NAME                 NETWORK  DIRECTION  PRIORITY  ALLOW     DENY  DISABLED
allow-imc-streamlit  default  INGRESS    1000      tcp:8501        False


In [ ]:
"""
countries_data = {
    "Poland": {
        "capital": "Warsaw",
        "population": 38_750_000,
        "area_km2": 312_696,
        "currency": "Złoty (PLN)",
        "eu_member": True,
        "language": "Polish"
    },
    "Germany": {
        "capital": "Berlin",
        "population": 84_120_000,
        "area_km2": 357_022,
        "currency": "Euro (EUR)",
        "eu_member": True,
        "language": "German"
    },
    "Ukraine": {
        "capital": "Kyiv",
        "population": 35_660_000,
        "area_km2": 603_628,
        "currency": "Hryvnia (UAH)",
        "eu_member": False,
        "language": "Ukrainian"
    }
}

def country_info_anna(country: str):
  info = countries_data.get(country)

  if not info:
    return {"error" : "Country not found",
            "Available countries":list(countries_data.keys())
    }

  return {
        "country": country,
        "data": info
    }
"""

"""
# --- Web UI con Streamlit
st.title("Chat with Gemini")
if "chat" not in st.session_state:
    st.session_state.chat = model.start_chat()
prompt = st.text_input("Ask me about Central Europe")
if st.button("Submit") and prompt.strip():
    st.chat_message("user").write(prompt)

    try:
        response = st.session_state.chat.send_message(prompt)
        part = response.candidates[0].content.parts[0]

        if part.function_call:
            fn_name = part.function_call.name
            fn_args = dict(part.function_call.args)

            if fn_name in tool_mapping:
                tool_result = tool_mapping[fn_name](**fn_args)

                response = st.session_state.chat.send_message(
                    Part.from_function_response(
                        name=fn_name,
                        response=tool_result,
                    )
                )

        st.chat_message("assistant").write(response.text)

    except Exception as e:
        st.error(f"Error: {e}")
        """"

In [16]:
%%writefile gemini-chat.py
import streamlit as st
import vertexai, google.auth
from vertexai.generative_models import (
    GenerativeModel,
    GenerationConfig,
    Part,
    Tool,
    FunctionDeclaration
)
import requests

def obtener_datos_pais(pais: str = "Polonia"):
    if pais.lower() not in ["polonia", "poland"]:
        return {"error": "Solo hay datos disponibles para Polonia"}
        
import requests

COUNTRY_COD = "POL"

urls = {
        "population": f"https://api.worldbank.org/v2/country/{COUNTRY_CODE}/indicator/SP.POP.TOTL?format=json",
        "gdp": f"https://api.worldbank.org/v2/country/{COUNTRY_CODE}/indicator/NY.GDP.MKTP.CD?format=json",
    }


# --- Diccionario para mapear el nombre de la función con la ejecución real
herramientas_mapeo = {
    "obtener_datos_pais": obtener_datos_pais
}

# --- Creación de las herramientas para el Modelo (Tool)
herramientas_agente = Tool(
    function_declarations = [
        FunctionDeclaration.from_func(obtener_datos_pais)
    ]
)


# --- Generar Modelo
credentials, project_id = google.auth.default()
vertexai.init(project=project_id, location="us-central1")

context = """
Eres un asistente experto en todo que concierne a pais centroeuropeo Polonia.
Tu objetivo es ayudar a turistas y locales.
Si realizan preguntas generales del pais {pais} (monumentos, fiestas, costumbres, ...), responde de forma correcta.
Si te preguntan por datos técnicos del pais {pais}, usa las herramientas disponibles. Presenta los datos de la siguiente forma:
DATOS DE POLONIA: 
- Habitantes: {info.poblacion}
- Superficie: {superficie_km2}
Si preguntan sobre otro contenido no relacionado con el {pais}, responde "No estoy programado para resolver esa pregunta".
"""
model_name = "gemini-2.5-flash-lite"
model = GenerativeModel(model_name, system_instruction = context)

st.title("Chat with Gemini")
if "chat" not in st.session_state and model:
    st.session_state.chat = model.start_chat()
    
if model:
    prompt = st.text_area("Introduce tu prompt:")
    if st.button("Generar Respuesta"):
        # --- Mostrar historial previo 
        if prompt.strip():
            with st.chat_message("user"):
                st.write(prompt)
            # ----------------------------------
            #  Lógica del Agente
            # ----------------------------------
            with st.chat_message("assistant"):
                with st.spinner("El agente está pensando..."):
                    try:
                        # --- Envío del mensaje al modelo
                        response = st.session_state.chat.send_message(prompt)
                        # --- Comprobación de si el Agente necesita usar una herramienta (Function Calling)
                        part = response.candidates[0].content.parts[0]
                        if part.function_call:
                            st.info(f"🛠️ El agente está usando la herramienta: `{part.function_call.name}`") 
                            # --- Extraer nombre de la función y argumentos
                            fn_name = part.function_call.name
                            fn_args = {key: val for key, val in part.function_call.args.items()}
                            # --- Ejecutar la función real de Python 
                            if fn_name in herramientas_mapeo:
                                api_response = herramientas_mapeo[fn_name](**fn_args)
                                # --- Devolver el resultado de la función al modelo para que genere la respuesta final
                                response = st.session_state.chat.send_message(
                                    Part.from_function_response(
                                        name=fn_name,
                                        response={
                                            "content": api_response,
                                        },
                                    )
                                )                            
                        # --- Mostrar la respuesta final (texto natural)
                        st.write(response.text)
                    except Exception as e:
                        st.error(f"Error en la lógica del agente: {str(e)}")
else:
    st.error("Error de inicialización. Revisa logs.")


Overwriting gemini-chat.py


In [5]:
#!streamlit run gemini-chat.py --server.port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.128.0.2:8501
  External URL: http://34.61.30.178:8501

/opt/conda/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
/opt/conda/lib/python3.10/site-packages/proto/marshal

In [4]:
!pip install --quiet streamlit